# Configuration

In [1]:
cd '/Users/xusikun/Desktop/Research/[research]tsinghua_options/options-study'

/Users/xusikun/Desktop/Research/[research]tsinghua_options/options-study


In [2]:
import os 
import scipy.io
import pandas as pd 
import numpy as np

from datetime import datetime

In [3]:
DATA = 'data/'

# Contracts

## Reader

In [4]:
mat = scipy.io.loadmat(os.path.join(DATA, 'DataFile.mat'))

In [5]:
mat.keys()
print('__header__', mat['__header__'])
print('__version__', mat['__version__'])
print('__globals__', mat['__globals__'])

__header__ b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Jun 11 17:36:09 2021'
__version__ 1.0
__globals__ []


## Build Contracts

In [6]:
from src.options_data import get_full_options_data

In [7]:
C = mat['Contract'].flatten()[0][0]
dates = [C[idx][0][0] for idx in range(C.shape[0])]
print(C.shape)

(1531, 21)


In [8]:
full = get_full_options_data(C, dates)

In [9]:
full[full['asof']=='2018-01-02']

,asof,code,name,type,strike,date,exp_date,time_to_exp
0,2018-01-02,10001073,50ETF购1月2896A,CALL,2.896,2017-11-23,2018-01-24,22 days
1,2018-01-02,10001074,50ETF购1月2946A,CALL,2.946,2017-11-23,2018-01-24,22 days
2,2018-01-02,10001075,50ETF购1月3044A,CALL,3.044,2017-11-23,2018-01-24,22 days
3,2018-01-02,10001076,50ETF购1月3142A,CALL,3.142,2017-11-23,2018-01-24,22 days
4,2018-01-02,10001077,50ETF购1月3240A,CALL,3.24,2017-11-23,2018-01-24,22 days
...,...,...,...,...,...,...,...,...
14,2018-01-02,10001124,50ETF沽6月3000,PUT,3,2017-11-28,2018-06-27,176 days
15,2018-01-02,10001132,50ETF沽6月2750,PUT,2.75,2017-12-01,2018-06-27,176 days
16,2018-01-02,10001140,50ETF沽6月2700,PUT,2.7,2017-12-18,2018-06-27,176 days
17,2018-01-02,10001169,50ETF沽6月2650,PUT,2.65,2018-01-02,2018-06-27,176 days


## Minute-level Data

In [12]:
from src.options_data import prepare_options_price_data_from_op510050, resample_options_data

In [13]:
folder1 = "/Volumes/Sikun's T7/data/[project]tsinghua_options_research/Op510050_1"
folder2 = "/Volumes/Sikun's T7/data/[project]tsinghua_options_research/Op510050_2"

options = prepare_options_price_data_from_op510050(
    filepath="/Volumes/Sikun's T7/data/[project]tsinghua_options_research/options.csv", 
    raw1=folder1, raw2=folder2, 
    save=True, save_filepath="/Volumes/Sikun's T7/data/[project]tsinghua_options_research/options.csv", 
)

In [15]:
daily_data = resample_options_data(options, freq='1D')

## Strangle

In [358]:
from src.strategy import get_daily_strangle

In [366]:
%%time
security_selected = []
for date in full['asof'].unique():
    daily = full[full['asof'] == date].reset_index(drop=True)
    security_selected += get_daily_strangle(
        date=date, 
        options_book=daily, 
        ETF=ETF
    )

CPU times: user 1min 23s, sys: 1.43 s, total: 1min 24s
Wall time: 1min 28s


In [367]:
pd.DataFrame(security_selected)

,asof,code,name,type,strike,date,exp_date,time_to_exp
0,2015-02-16,10000049.SH,50ETF购3月2500,CALL,2.5,2015-02-12,2015-03-25,37 days
1,2015-02-16,10000008.SH,50ETF沽3月2300,PUT,2.3,2015-02-09,2015-03-25,37 days
2,2015-02-16,10000051.SH,50ETF购4月2500,CALL,2.5,2015-02-12,2015-04-22,65 days
3,2015-02-16,10000018.SH,50ETF沽4月2300,PUT,2.3,2015-02-09,2015-04-22,65 days
4,2015-02-16,10000053.SH,50ETF购6月2500,CALL,2.5,2015-02-12,2015-06-24,128 days
...,...,...,...,...,...,...,...,...
12492,2021-06-11,10003426.SH,50ETF沽7月3400,PUT,3.4,2021-5-27,2021-7-28,47 days
12493,2021-06-11,10003208.SH,50ETF购9月3700,CALL,3.7,2021-1-28,2021-9-22,103 days
12494,2021-06-11,10003214.SH,50ETF沽9月3400,PUT,3.4,2021-1-28,2021-9-22,103 days
12495,2021-06-11,10003379.SH,50ETF购12月3700,CALL,3.7,2021-4-29,2021-12-22,194 days


***MISSNG OPTIONS PRICE DATA***

# ETF 

In [117]:
ETF = mat['Contract'].flatten()[0][1]
ETF = pd.DataFrame(
    ETF, 
    index=pd.to_datetime(np.array(dates)), 
    columns=['open', 'high', 'low', 'close', 'adj_close']
)
ETF

,open,high,low,close,adj_close
2015-02-16,2.390,2.403,2.379,2.394,2.3900
2015-02-17,2.401,2.429,2.401,2.411,2.4155
2015-02-25,2.411,2.411,2.359,2.370,2.3850
2015-02-26,2.368,2.452,2.348,2.450,2.4051
2015-02-27,2.447,2.465,2.428,2.438,2.4494
...,...,...,...,...,...
2021-06-07,3.604,3.606,3.570,3.593,3.5840
2021-06-08,3.591,3.630,3.550,3.572,3.5850
2021-06-09,3.570,3.590,3.562,3.582,3.5780
2021-06-10,3.582,3.630,3.577,3.601,3.6090


# IH

In [219]:
IH = mat['Contract'].flatten()[0][2]
print(IH.shape)

(1531, 14)
